<a href="https://colab.research.google.com/github/hanzlayyy/FYDP/blob/main/FYDP_Notebook_V1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder

In [9]:
df = pd.read_csv('/content/sample_data/data v1.csv')

In [ ]:
df

,layer_height,infill_density,infill_pattern,nozzle_temperature,print_speed,material,roughness
0,0.02,90.0,grid,220.0,40.0,abs,25
1,0.02,90.0,honeycomb,225.0,40.0,abs,32
2,0.02,80.0,grid,230.0,40.0,abs,40
3,0.02,70.0,honeycomb,240.0,40.0,abs,68
4,0.02,90.0,grid,250.0,40.0,abs,92
...,...,...,...,...,...,...,...
544,0.30,10.0,rectilinear,215.0,120.0,abs,1.64
545,0.30,10.0,rectilinear,215.0,120.0,abs,1.4
546,0.30,10.0,rectilinear,215.0,90.0,abs,0.82
547,0.30,10.0,rectilinear,215.0,90.0,abs,0.76


In [10]:
df=df.drop(403, axis=0)

In [12]:
df['infill_pattern'].unique()

array(['grid', 'honeycomb', 'tetrahedral', nan, 'zigzag', 'triangle',
       'concentric', 'line', 'line ', 'solid', 'rectilinear'],
      dtype=object)

In [11]:
df.isnull().mean()

layer_height          0.000000
infill_density        0.166058
infill_pattern        0.443431
nozzle_temperature    0.000000
print_speed           0.049270
material              0.049270
roughness             0.107664
dtype: float64

In [ ]:
df = df.replace('line ', 'line')

In [ ]:
missing = df[df.isnull().any(axis=1)]

In [ ]:
zero_missing = df[~df.isnull().any(axis=1)]

In [ ]:
one_missing_infill_density = missing[missing['infill_density'].isnull() & ~missing[['infill_pattern','print_speed','material','roughness']].isnull().any(axis=1)]


In [ ]:
one_missing_infill_pattern = missing[missing['infill_pattern'].isnull() & ~missing[['infill_density','print_speed','material','roughness']].isnull().any(axis=1)]


In [ ]:
one_missing_print_speed = missing[missing['print_speed'].isnull() & ~missing[['infill_density','infill_pattern','material','roughness']].isnull().any(axis=1)]


In [ ]:
one_missing_material = missing[missing['material'].isnull() & ~missing[['infill_density','infill_pattern','print_speed','roughness']].isnull().any(axis=1)]


In [ ]:
one_missing_roughness = missing[missing['roughness'].isnull() & ~missing[['infill_density','infill_pattern','print_speed','material']].isnull().any(axis=1)]


In [ ]:
combined = pd.concat([zero_missing, one_missing_infill_density], axis=0)
combined

,layer_height,infill_density,infill_pattern,nozzle_temperature,print_speed,material,roughness
0,0.02,90.0,grid,220.0,40.0,abs,25
1,0.02,90.0,honeycomb,225.0,40.0,abs,32
2,0.02,80.0,grid,230.0,40.0,abs,40
3,0.02,70.0,honeycomb,240.0,40.0,abs,68
4,0.02,90.0,grid,250.0,40.0,abs,92
...,...,...,...,...,...,...,...
357,0.15,NaN,grid,225.0,80.0,pla,16.17
358,0.15,NaN,grid,225.0,80.0,pla,17.12
359,0.15,NaN,concentric,225.0,80.0,pla,12.75
360,0.15,NaN,zigzag,225.0,80.0,pla,18.8


In [ ]:
temp = pd.get_dummies(combined, columns=['infill_pattern','material'], dtype=int)

,layer_height,infill_density,nozzle_temperature,print_speed,roughness,infill_pattern_concentric,infill_pattern_grid,infill_pattern_honeycomb,infill_pattern_line,infill_pattern_rectilinear,infill_pattern_solid,infill_pattern_tetrahedral,infill_pattern_triangle,infill_pattern_zigzag,material_abs,material_pla,material_pla-aluminum
0,0.02,90.0,220.0,40.0,25,0,1,0,0,0,0,0,0,0,1,0,0
1,0.02,90.0,225.0,40.0,32,0,0,1,0,0,0,0,0,0,1,0,0
2,0.02,80.0,230.0,40.0,40,0,1,0,0,0,0,0,0,0,1,0,0
3,0.02,70.0,240.0,40.0,68,0,0,1,0,0,0,0,0,0,1,0,0
4,0.02,90.0,250.0,40.0,92,0,1,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357,0.15,NaN,225.0,80.0,16.17,0,1,0,0,0,0,0,0,0,0,1,0
358,0.15,NaN,225.0,80.0,17.12,0,1,0,0,0,0,0,0,0,0,1,0
359,0.15,NaN,225.0,80.0,12.75,1,0,0,0,0,0,0,0,0,0,1,0
360,0.15,NaN,225.0,80.0,18.8,0,0,0,0,0,0,0,0,1,0,1,0
